<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


###IBAN Generator###

In [107]:
# IBAN Generator

import datetime
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)
print (eiban)





LC7200322800994985921


###Payment Generator###

In [204]:
# Erstellung von Transaktionsdaten 

from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# Bankbeziehungsdaten Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','IBAN','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 


# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)

# Betrugseinstufung 
# Kein Fraud ist immer 0
fraud_ids = [[1,'External Caused Fraud (e.g. Trojans)'],[2,'Internal Caused Fraud (e.g. Employee)']]
df_fraud = pd.DataFrame(fraud_ids, columns=['Fraud ID', 'Description'])


# Bankkunden Stammdaten - Datenquellen Github
custdb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/cutomer_base_data.csv'

cust_data = pd.read_csv(custdb_url, delimiter=';')
cust_data.columns=['anrede','vorname','nachname','strasse','nr','plz','ort',
                   'kanton','sprachregion',	'nation',	'sexcode','mobile_nr',	
                   'vertrag_nr','konto_nr','auth_device']

# Importierte Kundendaten bereinigen
cust_data = cust_data.fillna(0)
cust_data_dtype = {'plz':int,'mobile_nr':int,'vertrag_nr':int}
cust_data = cust_data.astype(cust_data_dtype)

# Erstellung der Zahlungs-Basisinformationen 

#start_date = datetime(2018,1,1,8,0,0)
trx_data = pd.DataFrame(columns=['timestamp','paym_id','send_bc_nr','amount',
                                 'rcv_bc_nr','rcv_bc_iban','rcv_bc_country','rcv_bc_code',
                                 'valuta_date','cust_vertrag_nr','cust_konto_nr',
                                 'cust_vname','cust_nname','cust_plz','cust_ort',
                                'cust_kanton','cust_sprachregion','cust_auth_device',
                                'fraud_id'])

for i in range(80):
    trx_data.loc[i,['timestamp']] = timestamps = pd.Timestamp(datetime(2018,1,1,8,0,0)+timedelta(seconds=30*i)).strftime("%Y.%m.%d %H:%M")
    trx_data.loc[i,['paym_id']] =  np.int64(np.random.randint(1000000000000,2000000000000,1))
    trx_data.loc[i,['send_bc_nr']] =  'CHBANK71XXX'
    trx_data.loc[i,['amount']] = np.int64(np.random.randint(100,100000,1)).astype(str)
    trx_data.loc[i,['rcv_bc_nr']] = bc_bank = np.random.choice(np.ravel(iban_cc['Bank']).astype(str))
    trx_data.loc[i,['rcv_bc_iban']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']])
    trx_data.loc[i,['rcv_bc_country']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Country']])
    trx_data.loc[i,['rcv_bc_code']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Code']])
    timestamps = datetime.strptime(timestamps,"%Y.%m.%d %H:%M")
    trx_data.loc[i,['valuta_date']] = valuta = timestamps + timedelta(hours=12*i)
    trx_data.loc[i,['cust_vertrag_nr']] = cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))
    trx_data.loc[i,['cust_konto_nr']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr']])
    trx_data.loc[i,['cust_vname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['vorname']])
    trx_data.loc[i,['cust_nname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['nachname']])
    trx_data.loc[i,['cust_plz']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['plz']])
    trx_data.loc[i,['cust_ort']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['ort']])
    trx_data.loc[i,['cust_kanton']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['kanton']])
    trx_data.loc[i,['cust_sprachregion']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['sprachregion']])
    trx_data.loc[i,['cust_auth_device']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['auth_device']])
    trx_data.loc[i,['fraud_id']] = 0
    

# Von allen durchgeführten Zahlungen hat das Fraud-Detection System 0.5% Betrugszahlungen erkannt. 
trx_data_fraud = trx_data.sample(frac=0.90)
trx_data_fraud.fraud_id = ''
test = np.random.choice(np.ravel(df_fraud['Fraud ID']).astype(str))


for x in range(len(trx_data_fraud)):
  trx_data_fraud.loc[x,['fraud_id']] = np.random.choice(np.ravel(df_fraud['Fraud ID']).astype(str))
  
trx_data_fraud.head(len(trx_data_fraud))
  
  





,timestamp,paym_id,send_bc_nr,amount,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device,fraud_id
22,2018.01.01 08:11,1471827643262,CHBANK71XXX,51759,ILBANK38XXX,IL020108380000002149431,Israel,IL,2018-01-12 08:11:00,72716,CH5300772857406471904,Anton,Huber,9103,Schwellbrunn,AR,de,mtan,1
74,2018.01.01 08:37,1508374887206,CHBANK71XXX,63562,DOBANK23XXX,DO22BCBH00000000011003290022,Dominikanische Republik,DO,2018-02-07 08:37:00,75948,CH5300772552097684232,Martin,Künzi,5000,Aarau,AG,de,mtan,
77,2018.01.01 08:38,1998757248260,CHBANK71XXX,39369,LBBANK47XXX,LB02001400000302300023018319,Libanon,LB,2018-02-08 20:38:00,66449,CH5300772888934238404,Ursula,Abegglen,5033,Buchs AG,AG,de,ptan,
57,2018.01.01 08:28,1489882618853,CHBANK71XXX,82590,ESBANK76XXX,ES1321000555370200853027,Spanien,ES,2018-01-29 20:28:00,69870,CH5300772081780850523,Matteo,Giovannoni,6702,Claro,TI,it,ptan,1
43,2018.01.01 08:21,1372915309078,CHBANK71XXX,98529,LTBANK49XXX,LT027300010134441147,Litauen,LT,2018-01-22 20:21:00,75420,CH5300772507626925782,Martin,Jeger,6006,Luzern,LU,de,mtan,2
0,2018.01.01 08:00,1768783399353,CHBANK71XXX,19707,MUBANK53XXX,MU03MCBL0901000001879025000USD,Mauritius,MU,2018-01-01 08:00:00,77039,CH5300772217494563095,Barbara,Ehrat,9043,Trogen,AR,de,ptan,2
35,2018.01.01 08:17,1305586533158,CHBANK71XXX,5456,NOBANK59XXX,NO0239916835985,Norwegen,NO,2018-01-18 20:17:00,80497,CH5300772416924671408,Monika,Rutishauser,9104,Waldstatt,AR,de,ptan,2
65,2018.01.01 08:32,1314716461557,CHBANK71XXX,60718,LUBANK50XXX,LU020019175546294000,Luxemburg,LU,2018-02-02 20:32:00,69746,CH5300772892101859086,Ruth,Meier,9035,Grub AR,AR,de,ptan,2
45,2018.01.01 08:22,1076701633023,CHBANK71XXX,66344,GIBANK30XXX,GI04BARC020452163087000,Gibraltar,GI,2018-01-23 20:22:00,77881,CH5300772887644426819,Peter,Tönz,6245,Ebersecken,LU,de,mtan,2
5,2018.01.01 08:02,1424111818602,CHBANK71XXX,25724,VGBANK18XXX,VG48NOSC0000000005002993,Britische Jungferninseln,VG,2018-01-03 20:02:00,75575,CH5300772558726353113,Christian,Cantieni,6280,Hochdorf,LU,de,ptan,2


In [149]:


import pandas as pd
from random import randint

df = pd.DataFrame(columns=['A','B','C','D','E'])

for i in range(5): #add 5 rows of data
    df.loc[i, ['A']] = randint(0,99)
    df.loc[i, ['B']] = randint(0,99)
    df.loc[i, ['C']] = randint(0,99)
    df.loc[i, ['D']] = randint(0,99)
    df.loc[i, ['E']] = randint(0,99)
    
df

,A,B,C,D,E
0,82.0,93.0,1.0,5.0,8.0
1,4.0,34.0,45.0,13.0,88.0
2,87.0,41.0,54.0,41.0,43.0
3,81.0,44.0,98.0,3.0,83.0
4,47.0,52.0,20.0,45.0,60.0
